#Using Llama-3-8B-Instruct to answer MMLU questions

In [ ]:
!pip install accelerate -U
!pip install tqdm

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from tqdm import tqdm
from google.colab import files
import pandas as pd
import io

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

access_token='hf_access_token'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", token=access_token)
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", token=access_token)


In [ ]:
def create_prompt(question, possible_answers):
    prompt = f"Answer the following question by selecting the most appropriate choice.\nQuestion: {question}\n"
    prompt += "Choices:\n"
    for idx, answer in enumerate(possible_answers):
        prompt += f"{idx}. {answer}\n"
    prompt += "Answer (return only the corresponding number):"
    return prompt


def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_new_tokens=3,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.01,
            do_sample=False
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


In [ ]:
uploaded = files.upload()

df = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

In [ ]:
# Add a progress bar to pandas apply
tqdm.pandas()

df['model_response'] = df.progress_apply(
    lambda row: generate_response(create_prompt(row['prompt'], [row['0'], row['1'], row['2'], row['3']]))[-2],
    axis=1
)

def check_correctness(model_response, correct_answer, possible_answers):
    if model_response == "\n":
      return 0
    try:
      model_response = int(model_response)
    except:
      return 0
    correct_answer = int(correct_answer)
    # Check if the model's response is one of the possible answers
    return int(model_response in possible_answers and model_response == correct_answer)

df['is_correct'] = df.apply(lambda row: check_correctness(row['model_response'], row['answer'], [0,1,2,3]), axis=1)
accuracy = df['is_correct'].mean()
print(f"Model Accuracy: {accuracy * 100:.2f}%")

output_csv_path = "dataset_results.csv"
df.to_csv(output_csv_path, index=False)

#Generating Llama Rephrases

In [ ]:
def generate_response(prompt):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_new_tokens=300,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            temperature=0.01,
            do_sample=False
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response


def generate_new_prompt(question, possible_answers, idx):
    dict_idx_to_do = {"4": "question", "0": "first answer", "1": "second answer", "2": "third answer", "3": "fourth answer"}
    prompt = f"You are given the following multichoice question with the following answers, please rephrase the {dict_idx_to_do[idx]}. Make sure your answer is between 50-250 charachters.\nQuestion: {question}\n"
    prompt += "Choices:\n"
    for idx2, answer in enumerate(possible_answers):
        prompt += f"{idx2}. {answer}\n"
    return prompt

def get_generations():
    df['new_question'] = df.progress_apply(
    lambda row: generate_response(generate_new_prompt(row['prompt'], [row['0'], row['1'], row['2'], row['3']], 4)),
    axis=1
)
    df['0_new'] = df.progress_apply(
    lambda row: generate_response(generate_new_prompt(row['prompt'], [row['0'], row['1'], row['2'], row['3']], 0)),
    axis=1
)
    df['1_new'] = df.progress_apply(
    lambda row: generate_response(generate_new_prompt(row['prompt'], [row['0'], row['1'], row['2'], row['3']], 1)),
    axis=1
)
    df['2_new'] = df.progress_apply(
    lambda row: generate_response(generate_new_prompt(row['prompt'], [row['0'], row['1'], row['2'], row['3']], 2)),
    axis=1
)
    df['3_new'] = df.progress_apply(
    lambda row: generate_response(generate_new_prompt(row['prompt'], [row['0'], row['1'], row['2'], row['3']], 3)),
    axis=1
)

In [ ]:
get_generations()

In [ ]:
output_csv_path = "llama_generations_dataset.csv"
df.to_csv(output_csv_path, index=False)